# Case Study: ARIMA Modeling for Energy Data

In this notebook, we do a back of the envelope case study to see if we can accurately model energy demand

Our goal will be to use the data from 2017 to 2020 and see whether we can accurately predict energy demand for _all_ of 2021!

In [ ]:
import requests
import zipfile

from io import BytesIO

import matplotlib.pyplot as plt
import numba
import numpy as np
import pandas as pd

from sklearn.linear_model import LinearRegression
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA

%matplotlib inline

## Data

We will return to our ERCOT load data.

Recall that this data represents the hourly energy demand in various regions of Texas.

We will aggregate it to daily totals and try to predict daily totals

In [ ]:
# Run this code if you need to download the data
# url = (
#     "http://www.ercot.com/content/wcm"
#     "/key_documents_lists/89022/"
# )
# fn = "Native_Load_{year}.zip"

# load_dfs = []
# for year in range(2017, 2021):
#     if year < 2019:
#         fn_to_use = fn.lower().format(year=str(year))
#     else:
#         fn_to_use = fn.format(year=str(year))
#     url_to_use = url + fn_to_use

#     res = requests.get(url_to_use)
#     zf = zipfile.ZipFile(BytesIO(res.content))
#     load_dfs.append(
#         pd.read_excel(
#             zf.read(zf.filelist[0].filename),
#             engine="openpyxl"
#         ).rename(
#             columns={
#                 "HourEnding": "dt",
#                 "Hour Ending": "dt"
#             }
#         )
#     )

# load = pd.concat(load_dfs)

# # Convert hours from 1-24 to 0-23
# for i in range(24):
#     old, new = f"{i+1:02}:00", f"{i:02}:00"
#     load["dt"] = load["dt"].str.replace(old, new)
# load["dt"] = pd.to_datetime(load["dt"])
# load = load.set_index("dt").resample("D").sum()

# load.head()
# load.to_parquet("load_2017_2020.parquet")

In [ ]:
load = pd.read_parquet("load_2017_2020.parquet")

load.head()

In [ ]:
years = load.index.year.unique().values

fig, ax = plt.subplots(years.size, figsize=(14, 8))
for (iy, year) in enumerate(years):
    bs = ((load.index.year == year))
    ax[iy].plot(load.loc[bs, "ERCOT"])


## Computing the trend

We will assume a linear (in time) trend

In [ ]:
nobs = load.shape[0]
trend_tvalues = np.arange(nobs)

mod_trend = LinearRegression(fit_intercept=True)
mod_trend.fit(trend_tvalues[:, None], load["ERCOT"])

load["ERCOT_trend"] = mod_trend.predict(trend_tvalues[:, None])
load["ERCOT_trend_removed"] = load.eval("ERCOT - ERCOT_trend")

In [ ]:
fig, ax = plt.subplots(2, figsize=(10, 8), sharex=True)

load.plot(y="ERCOT", ax=ax[0], color="k", legend=False)
load.plot(y="ERCOT_trend", ax=ax[0], color="DarkOrange", linestyle="--", legend=False)
ax[0].set_title("ERCOT and ERCOT Trend")

load.plot(y="ERCOT_trend_removed", ax=ax[1], legend=False)
ax[1].set_title("Trend Removed")

fig.tight_layout()

## Computing seasonal elements

For the monthly and day of week seasonality, we will use the seasonal fixed effects model

**Monthly seasonality**

In [ ]:
# Get fixed effect values for each "season" of our choosing
months = list(range(1, 13))
month_cols = [f"month_{i}" for i in months]

for month in month_cols:
    _, i = month.split("_")
    i = int(i)
    load[month] = (load.index.month == i).astype(int)


In [ ]:
mod_seas_m = LinearRegression(fit_intercept=False)
mod_seas_m.fit(load.loc[:, month_cols], load["ERCOT_trend_removed"])

load["ERCOT_mseasonal"] = mod_seas_m.predict(load.loc[:, month_cols])
load["ERCOT_trend_mseasonal"] = load.eval("ERCOT_trend + ERCOT_mseasonal")
load["ERCOT_trend_mseasonal_removed"] = load.eval("ERCOT_trend_removed - ERCOT_mseasonal")

In [ ]:
fig, ax = plt.subplots(2, figsize=(10, 8), sharex=True)

load.plot(y="ERCOT", ax=ax[0], color="k", legend=False)
load.plot(
    y="ERCOT_trend_mseasonal", ax=ax[0], color="DarkOrange",
    linestyle="--", legend=False, linewidth=3
)
ax[0].set_title("ERCOT and ERCOT Seasonal")

load.plot(y="ERCOT_trend_mseasonal_removed", ax=ax[1], legend=False)
ax[1].set_title("Trend and Monthly Season Removed")

fig.tight_layout()

**Daily seasonality**

In [ ]:
# Get fixed effect values for each "season" of our choosing
dows = list(range(7))
dow_cols = [f"day_{i}" for i in dows]

for dow in dow_cols:
    _, i = dow.split("_")
    i = int(i)

    load[dow] = (load.index.dayofweek == i).astype(int)


In [ ]:
mod_seas_d = LinearRegression(fit_intercept=False)
mod_seas_d.fit(load.loc[:, dow_cols], load["ERCOT_trend_mseasonal_removed"])

load["ERCOT_dseasonal"] = mod_seas_d.predict(load.loc[:, dow_cols])
load["ERCOT_trend_mdseasonal"] = load.eval("ERCOT_trend_mseasonal + ERCOT_dseasonal")
load["ERCOT_trend_mdseasonal_removed"] = load.eval("ERCOT_trend_mseasonal_removed - ERCOT_dseasonal")

In [ ]:
fig, ax = plt.subplots(2, figsize=(10, 8), sharex=True)

load.plot(y="ERCOT", ax=ax[0], color="k", legend=False)
load.plot(y="ERCOT_trend_mdseasonal", ax=ax[0], color="DarkOrange", linestyle="--", legend=False)
ax[0].set_title("ERCOT and ERCOT Seasonal")

load.plot(y="ERCOT_trend_mdseasonal_removed", ax=ax[1], legend=False)
ax[1].set_title("Trend and Monthly Season Removed")

fig.tight_layout()

## Fitting an ARMA model to remainder

In [ ]:
plot_acf(load.loc[:, "ERCOT_trend_mdseasonal_removed"]);

In [ ]:
plot_pacf(load.loc[:, "ERCOT_trend_mdseasonal_removed"]);

In [ ]:
arma = ARIMA(load.loc[:, "ERCOT_trend_mdseasonal_removed"], order=(3, 0, 3))
arma_res = arma.fit()

print(arma_res.summary())

## Forecasting 2021

In [ ]:
# url_to_use = "http://www.ercot.com/content/wcm/key_documents_lists/89022/Native_Load_2021.zip"

# res = requests.get(url_to_use)
# zf = zipfile.ZipFile(BytesIO(res.content))
# load_2021 = pd.read_excel(
#     zf.read(zf.filelist[0].filename),
#     engine="openpyxl"
# ).rename(
#     columns={
#         "HourEnding": "dt",
#         "Hour Ending": "dt"
#     }
# )

# # Convert hours from 1-24 to 0-23
# for i in range(24):
#     old, new = f"{i+1:02}:00", f"{i:02}:00"
#     load_2021["dt"] = load_2021["dt"].str.replace(old, new)
# load_2021["dt"] = pd.to_datetime(load_2021["dt"])
# load_2021 = load_2021.set_index("dt").resample("D").sum()

# load_2021.to_parquet("load_2021.parquet")

In [ ]:
load_2021 = pd.read_parquet("load_2021.parquet")
nobs_2021 = load_2021.shape[0]

load_2021.head()

**Predicting each component**

In [ ]:
arma_res_forecast = arma_res.get_forecast(nobs_2021)
arma_forecast = arma_res_forecast.predicted_mean

In [ ]:
trend_tvalues_forecast = (trend_tvalues[-1] + 1) + np.arange(load_2021.shape[0])

trend_forecast = mod_trend.predict(trend_tvalues_forecast[:, None])

In [ ]:
# Get fixed effect values for each month
for month in month_cols:
    _, i = month.split("_")
    i = int(i)

    load_2021[month] = (load_2021.index.month == i).astype(int)

month_forecast = mod_seas_m.predict(load_2021.loc[:, month_cols])

In [ ]:
# Get fixed effect values for each day of week
for dow in dow_cols:
    _, i = dow.split("_")
    i = int(i)

    load_2021[dow] = (load_2021.index.dayofweek == i).astype(int)

dow_forecast = mod_seas_d.predict(load_2021.loc[:, dow_cols])

In [ ]:
load_2021["ERCOT_forecast"] = trend_forecast + month_forecast + dow_forecast + arma_forecast

fig, ax = plt.subplots(figsize=(10, 8))

load_2021.plot(y="ERCOT", ax=ax, color="k")
load_2021.plot(y="ERCOT_forecast", ax=ax, color="DarkOrange", linestyle="--")

ax.annotate("Realized Load", xy=(18630, 1250000))
ax.annotate("Forecasted Load", xy=(18690, 1000000), color="DarkOrange")

ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))

load.loc["2020-01-01":"2020-04-15", :].plot(
    y="ERCOT", ax=ax, color="k", legend=False
)
load.loc["2020-01-01":"2020-04-15", :].plot(
    y="ERCOT_trend_mdseasonal", ax=ax,
    color="DarkOrange", linestyle="--", legend=False
)

ax.annotate("Realized Load", xy=(18350, 1050000))
ax.annotate("Forecasted Load", xy=(18265, 1030000), color="DarkOrange")

ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)

In [ ]:
ax.get_xlim()